<a href="https://colab.research.google.com/github/OzasaHiro/NSFK/blob/main/yt_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
youtube_link_to_analyze = 'https://www.youtube.com/watch?v=dQw4w9WgXcQ'

In [ ]:
def extract_frames_and_audio(video_filepath, temp_dir, frames_per_second=0.5):
    """
    Extracts frames and audio from a video file, with adaptive sampling
    based on scene changes and a base interval.

    Args:
        video_filepath (str): Path to the input video file.
        temp_dir (str): Temporary directory to save extracted files.
        frames_per_second (float): Base rate of frames to extract per second of video.
                                  Scene changes will also be included.

    Returns:
        tuple: (list of base64 encoded image strings, path to extracted audio file)
               Returns ([], None) if an error occurs.
    """
    if not os.path.exists(video_filepath):
        print(f"Error: Video file not found at '{video_filepath}'")
        return [], None

    cap = cv2.VideoCapture(video_filepath)
    if not cap.isOpened():
        print(f"Error: Could not open video file '{video_filepath}'")
        return [], None

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    video_duration_seconds = total_frames / fps

    print(f"Video Info: Total frames={total_frames}, FPS={fps:.2f}, Duration={video_duration_seconds:.2f}s")

    # --- Scene Detection ---
    print("\nDetecting scene changes...")
    scene_list = []
    try:
        # Use the updated detect_scenes function directly from scenedetect
        # It takes the video path and a list of detectors.
        # frame_skip=0 ensures every frame is processed for scene detection accuracy.
        scene_list = detect_scenes(video_filepath, [ContentDetector(threshold=27.0)], frame_skip=0)
        print(f"Detected {len(scene_list)} scenes.")
    except Exception as e:
        print(f"Error during scene detection: {e}. Proceeding without scene-based sampling.")
        scene_list = [] # Fallback if scene detection fails


    scene_change_frame_numbers = set()
    for scene in scene_list:
        # Add the start and end frames of each detected scene
        scene_change_frame_numbers.add(scene[0].frame_number)
        scene_change_frame_numbers.add(scene[1].frame_number)
        # Also add a few frames around the scene change for more context
        for offset in [-int(fps/2), -1, 1, int(fps/2)]: # e.g., 0.5 sec before/after, and immediate frames
             if 0 <= (scene[0].frame_number + offset) < total_frames:
                 scene_change_frame_numbers.add(scene[0].frame_number + offset)
             if 0 <= (scene[1].frame_number + offset) < total_frames:
                 scene_change_frame_numbers.add(scene[1].frame_number + offset)


    # --- Frame Sampling and Conversion ---
    cap = cv2.VideoCapture(video_filepath)
    if not cap.isOpened():
        print(f"Error: Could not re-open video file '{video_filepath}' for frame extraction.")
        return [], None

    sampled_frames_data = {} # Use a dict to store {frame_number: base64_image} to avoid duplicates and maintain order

    # Calculate interval for base sampling
    frame_interval = int(fps / frames_per_second) if frames_per_second > 0 else 1
    if frame_interval == 0:
        frame_interval = 1 # Ensure at least one frame is picked if frames_per_second is very low

    current_frame_number = 0
    success, image = cap.read()

    print(f"Extracting frames (base: every {frame_interval}th frame, plus scene changes)...")
    while success:
        # Include frame if it's on the regular interval OR if it's a scene change frame
        if (current_frame_number % frame_interval == 0) or (current_frame_number in scene_change_frame_numbers):
            # Encode frame to JPEG
            _, buffer = cv2.imencode('.jpg', image)
            # Base64 encode and store, using the frame number as key to avoid duplicates
            sampled_frames_data[current_frame_number] = base64.b64encode(buffer).decode('utf-8')

        success, image = cap.read()
        current_frame_number += 1
    cap.release()

    # Sort frames by their number to maintain chronological order
    sorted_frame_numbers = sorted(sampled_frames_data.keys())
    final_sampled_frames_base64 = [sampled_frames_data[fn] for fn in sorted_frame_numbers]

    print(f"Total unique frames extracted: {len(final_sampled_frames_base64)}")

    # --- Audio Extraction ---
    audio_output_filepath = os.path.join(temp_dir, "extracted_audio.wav")
    print(f"Extracting audio to: {audio_output_filepath}")
    try:
        audio = AudioSegment.from_file(video_filepath)
        audio.export(audio_output_filepath, format="wav")
        print("Audio extraction complete.")
        return final_sampled_frames_base64, audio_output_filepath
    except Exception as e:
        print(f"Error extracting audio: {e}")
        print("Ensure FFmpeg is installed and accessible in your system's PATH for audio extraction.")
        return final_sampled_frames_base64, None

In [ ]:
def detect_scene_changes_opencv(video_filepath, threshold=2000000):
    """
    Detects scene changes in a video using simple frame differencing with OpenCV.
    A scene change is detected if the sum of absolute differences between
    consecutive grayscale frames exceeds a threshold.

    Args:
        video_filepath (str): Path to the input video file.
        threshold (int): The threshold for detecting a scene change.
                         Higher value means less sensitive detection.
                         Adjust based on video characteristics.

    Returns:
        set: A set of frame numbers where scene changes are detected.
    """
    print("Detecting scene changes using OpenCV frame differencing...")
    scene_change_frames = set()
    cap = cv2.VideoCapture(video_filepath)
    if not cap.isOpened():
        print(f"Error: Could not open video for scene detection: {video_filepath}")
        return scene_change_frames

    ret, prev_frame = cap.read()
    if not ret:
        cap.release()
        return scene_change_frames

    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    current_frame_number = 0

    while True:
        ret, current_frame = cap.read()
        if not ret:
            break # End of video

        current_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)

        # Calculate the absolute difference between the current and previous grayscale frames
        frame_diff = cv2.absdiff(current_gray, prev_gray)

        # Sum up all pixel differences. If it's a large value, a scene change likely occurred.
        diff_sum = np.sum(frame_diff)

        if diff_sum > threshold:
            # Mark the current frame as a scene change
            scene_change_frames.add(current_frame_number)
            # Also add the frame just before the cut for context
            if current_frame_number > 0:
                scene_change_frames.add(current_frame_number - 1)

        prev_gray = current_gray
        current_frame_number += 1

    cap.release()
    print(f"Detected {len(scene_change_frames)} scene changes.")
    return scene_change_frames


def extract_frames_and_audio(video_filepath, temp_dir, frames_per_second=0.5, scene_threshold=2000000):
    """
    Extracts frames and audio from a video file, with adaptive sampling
    based on scene changes and a base interval.

    Args:
        video_filepath (str): Path to the input video file.
        temp_dir (str): Temporary directory to save extracted files.
        frames_per_second (float): Base rate of frames to extract per second of video.
                                  Scene changes will also be included.
        scene_threshold (int): Threshold for OpenCV scene detection.

    Returns:
        tuple: (list of base64 encoded image strings, path to extracted audio file)
               Returns ([], None) if an error occurs.
    """
    if not os.path.exists(video_filepath):
        print(f"Error: Video file not found at '{video_filepath}'")
        return [], None

    cap = cv2.VideoCapture(video_filepath)
    if not cap.isOpened():
        print(f"Error: Could not open video file '{video_filepath}'")
        return [], None

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    video_duration_seconds = total_frames / fps

    print(f"Video Info: Total frames={total_frames}, FPS={fps:.2f}, Duration={video_duration_seconds:.2f}s")

    # --- Scene Detection ---
    # Call the custom OpenCV scene detection function
    scene_change_frame_numbers = detect_scene_changes_opencv(video_filepath, threshold=scene_threshold)

    # --- Frame Sampling and Conversion ---
    # Re-open video capture as scene detection might have closed it
    cap = cv2.VideoCapture(video_filepath)
    if not cap.isOpened():
        print(f"Error: Could not re-open video file '{video_filepath}' for frame extraction.")
        return [], None

    sampled_frames_data = {} # Use a dict to store {frame_number: base64_image} to avoid duplicates and maintain order

    # Calculate interval for base sampling
    frame_interval = int(fps / frames_per_second) if frames_per_second > 0 else 1
    if frame_interval == 0:
        frame_interval = 1 # Ensure at least one frame is picked if frames_per_second is very low

    current_frame_number = 0
    success, image = cap.read()

    print(f"Extracting frames (base: every {frame_interval}th frame, plus {len(scene_change_frame_numbers)} scene changes)...")
    while success:
        # Include frame if it's on the regular interval OR if it's a scene change frame
        if (current_frame_number % frame_interval == 0) or (current_frame_number in scene_change_frame_numbers):
            # Encode frame to JPEG
            _, buffer = cv2.imencode('.jpg', image)
            # Base64 encode and store, using the frame number as key to avoid duplicates
            sampled_frames_data[current_frame_number] = base64.b64encode(buffer).decode('utf-8')

        success, image = cap.read()
        current_frame_number += 1
    cap.release()

    # Sort frames by their number to maintain chronological order
    sorted_frame_numbers = sorted(sampled_frames_data.keys())
    final_sampled_frames_base64 = [sampled_frames_data[fn] for fn in sorted_frame_numbers]

    print(f"Total unique frames extracted: {len(final_sampled_frames_base64)}")

    # --- Audio Extraction ---
    audio_output_filepath = os.path.join(temp_dir, "extracted_audio.wav")
    print(f"Extracting audio to: {audio_output_filepath}")
    try:
        audio = AudioSegment.from_file(video_filepath)
        audio.export(audio_output_filepath, format="wav")
        print("Audio extraction complete.")
        return final_sampled_frames_base64, audio_output_filepath
    except Exception as e:
        print(f"Error extracting audio: {e}")
        print("Ensure FFmpeg is installed and accessible in your system's PATH for audio extraction.")
        return final_sampled_frames_base64, None


In [ ]:
!pip install whisper
!pip install scenedetect # Install scenedetect


import os
import cv2 # For video frame extraction
import tempfile
import base64
import json
import asyncio
from pydub import AudioSegment # For audio extraction/conversion
import whisper # For Speech-to-Text
import numpy as np # For numerical operations with OpenCV frames

# For scene detection
from scenedetect import VideoManager
from scenedetect import SceneManager
from scenedetect.detectors import ContentDetector

# For actual Gemini API calls - Changing from 'requests' to 'aiohttp' for async calls
import aiohttp # Install with: pip install aiohttp

# Import nest_asyncio to handle running event loops in interactive environments
try:
    import nest_asyncio
    nest_asyncio.apply()
except ImportError:
    print("nest_asyncio not found. If you encounter 'RuntimeError: Event loop is already running',"
          " consider installing it: pip install nest_asyncio")


# --- Configuration ---
# You need a valid API key for gemini-2.0-flash.
# In the Canvas environment, __api_key will be automatically provided if left empty.
API_KEY = YOUR_YOUTUBE_API_KEY
GEMINI_API_URL = YOUR_GEMINI_API_KEY

def transcribe_audio_with_whisper(audio_file_path):
    """
    Transcribes an audio file using the OpenAI Whisper model.
    To speed up: consider using a smaller model ('tiny', 'base') or
    ensure your system can leverage GPU if using 'openai-whisper' (PyTorch version).
    """
    if not audio_file_path or not os.path.exists(audio_file_path):
        print("No audio file for transcription or file not found.")
        return ""

    print("\nLoading Whisper model (this might take a moment)...")
    try:
        # Choose a smaller model like 'tiny' or 'base' for faster transcription
        # e.g., model = whisper_cpp.Whisper.from_pretrained("tiny")
        model = whisper.load_model("base")
    except Exception as e:
        print(f"Error loading Whisper model: {e}")
        print("Please ensure whisper-cpp-python is correctly installed and can download model files.")
        return ""

    print("Transcribing audio...")
    try:
        result = model.transcribe(audio_file_path)
        transcription_text = result["text"]
        print("Transcription complete.")
        return transcription_text
    except Exception as e:
        print(f"Error during transcription: {e}")
        return ""

async def analyze_frame_with_gemini(session, base64_image, frame_number, video_title):
    """
    Sends a single base64 encoded image to Gemini 2.0 Flash for safety analysis asynchronously.
    """
    headers = {
        'Content-Type': 'application/json',
    }

    # Prompt specific to identify unsafe content for children
    prompt_text = (
        f"Analyze this image from the video titled '{video_title}'. "
        f"Specifically describe anything that could be potentially offensive, inappropriate, or unsafe "
        f"for young children (e.g., violence, weapons, nudity, explicit content, drug/alcohol use, "
        f"dangerous situations, scary imagery, jump scares, disturbing visual effects). "
        f"If the image appears completely safe and normal for kids, respond with 'SAFE_FOR_KIDS'."
    )

    payload = {
        "contents": [
            {
                "role": "user",
                "parts": [
                    {"text": prompt_text},
                    {"inlineData": {"mimeType": "image/jpeg", "data": base64_image}}
                ]
            }
        ]
    }

    try:
        async with session.post(f"{GEMINI_API_URL}?key={API_KEY}", headers=headers, json=payload) as response:
            response.raise_for_status() # Raise an exception for HTTP errors (4xx or 5xx)
            result = await response.json()

            if result.get('candidates') and result['candidates'][0].get('content') and result['candidates'][0]['content'].get('parts'):
                description = result['candidates'][0]['content']['parts'][0]['text'].strip()
                if description == "SAFE_FOR_KIDS":
                    # print(f"  Frame {frame_number}: Safe.") # Keep quiet for safe frames to reduce console spam
                    return None # No unsafe content found for this frame
                else:
                    print(f"  Frame {frame_number}: Potential Issue: {description[:100]}...") # Print first 100 chars
                    return f"Frame {frame_number}: {description}"
            else:
                print(f"  Frame {frame_number}: No valid response from LLM.")
                return f"Frame {frame_number}: (Frame {frame_number} - No specific analysis from LLM, check response structure)"
    except aiohttp.ClientError as e:
        print(f"  Frame {frame_number}: API request failed: {e}")
        # Correct indentation for the return statement
        return f"Frame {frame_number}: (Frame {frame_number} - API request failed: {e})"
    except json.JSONDecodeError:
        print(f"  Frame {frame_number}: Failed to decode JSON from LLM response.")
        # Correct indentation for the return statement
        return f"Frame {frame_number}: (Frame {frame_number} - JSON decode error from LLM)"
    except Exception as e:
        print(f"  Frame {frame_number}: Unexpected error during LLM analysis: {e}")
        # Correct indentation for the return statement
        return f"Frame {frame_number}: (Frame {frame_number} - Unexpected LLM analysis error: {e})"


async def video_safety_agent(video_filepath, video_title):
    """
    Analyzes a local video's visuals and audio for child safety,
    summarizing potentially offensive or unsafe content for parents.

    Args:
        video_filepath (str): The full path to the locally downloaded video file.
        video_title (str): The title of the video (for context in analysis).

    Returns:
        str: A comprehensive safety summary for parents.
    """
    print(f"--- Starting Video Safety Analysis for: '{video_title}' ---")
    print(f"Video File: {video_filepath}")

    # Use a temporary directory for all intermediate files
    with tempfile.TemporaryDirectory() as temp_dir:
        print(f"Using temporary directory: {temp_dir}")

        # Step 1: Extract frames and audio
        # Adjust frames_per_second for base sampling rate.
        # Scene change detection will add more frames where visual content changes.
        sampled_frames_base64, audio_filepath = extract_frames_and_audio(video_filepath, temp_dir, frames_per_second=0.5)

        # Step 2: Transcribe audio
        audio_transcript = transcribe_audio_with_whisper(audio_filepath)
        if not audio_transcript.strip():
            print("Warning: No audio transcript generated or it's empty.")

        # Step 3: Analyze sampled frames with Multimodal LLM concurrently
        print("\n--- Analyzing video frames for unsafe content concurrently ---")
        visual_safety_tasks = []

        # Use aiohttp.ClientSession for efficient concurrent HTTP requests
        async with aiohttp.ClientSession() as session:
            for i, frame_data in enumerate(sampled_frames_base64):
                # Create a task for each frame analysis
                task = analyze_frame_with_gemini(session, frame_data, i + 1, video_title)
                visual_safety_tasks.append(task)

            # Run all analysis tasks concurrently and wait for them to complete
            # This is the main speedup for LLM calls by parallelizing API requests.
            visual_safety_observations = await asyncio.gather(*visual_safety_tasks)
            # Filter out None values (where frames were deemed SAFE_FOR_KIDS)
            visual_safety_observations = [obs for obs in visual_safety_observations if obs is not None]

        # Step 4: Aggregate all information and summarize with LLM
        print("\n--- Aggregating information for final safety summary ---")

        combined_analysis_text = f"Video Title: '{video_title}'\n\n"
        combined_analysis_text += "--- Audio Transcript ---\n"
        if audio_transcript:
            combined_analysis_text += audio_transcript + "\n\n"
        else:
            combined_analysis_text += "No audio transcript available.\n\n"

        combined_analysis_text += "--- Visual Observations from Sampled Frames ---\n"
        if visual_safety_observations:
            for obs in visual_safety_observations:
                combined_analysis_text += obs + "\n"
        else:
            combined_analysis_text += "No specific unsafe visual content detected in sampled frames.\n"

        # Final prompt to the LLM for comprehensive safety assessment
        final_summary_prompt = (
            "Given the following information about a video, provide a comprehensive summary for parents "
            "indicating whether this video is safe for young children. "
            "Specifically highlight any content (visual or audio) that could be offensive, inappropriate, "
            "or dangerous, and explain why. If no such content is found, state that it appears generally safe. "
            "Consider themes of violence, explicit language, nudity, drug/alcohol use, scary imagery, "
            "or depictions of dangerous situations, jump scares, disturbing visual effects, or rapid flashing lights. "
            "Prioritize clear, concise language for parents, using bullet points for identified issues.\n\n"
            "Here is the collected data:\n"
            f"{combined_analysis_text}"
        )

        print("\n--- Final LLM call for overall safety assessment ---")
        headers = {
            'Content-Type': 'application/json',
        }
        payload = {
            "contents": [
                {
                    "role": "user",
                    "parts": [
                        {"text": final_summary_prompt}
                    ]
                }
            ]
        }

        try:
            # Using aiohttp for the final summary call as well for consistency
            async with aiohttp.ClientSession() as session:
                async with session.post(f"{GEMINI_API_URL}?key={API_KEY}", headers=headers, json=payload) as response:
                    response.raise_for_status()
                    result = await response.json()

                    if result.get('candidates') and result['candidates'][0].get('content') and result['candidates'][0]['content'].get('parts'):
                        final_summary = result['candidates'][0]['content']['parts'][0]['text'].strip()
                        return final_summary
                    else:
                        return "Error: Could not get a valid final summary from the LLM."
        except aiohttp.ClientError as e:
            return f"Error: Final LLM API request failed: {e}"
        except Exception as e:
            return f"Error: Unexpected error during final LLM summary: {e}"

# --- Main execution block ---
if __name__ == "__main__":
    # --- IMPORTANT: Replace this with the actual path to your downloaded video file ---
    # Example: local_video_path = "C:/Users/YourUser/Videos/my_downloaded_youtube_video.mp4"
    # Example: local_video_path = "/Users/YourUser/Downloads/my_downloaded_youtube_video.webm"
    local_video_path = input("Enter the full path to your downloaded YouTube video file: ")

    # You might get the title from a filename or the YouTube Data API if needed
    # For simplicity, we'll extract it from the filename for this example
    # video_filename = os.path.basename(local_video_path)
    # video_title_from_file = os.path.splitext(video_filename)[0] # Remove extension for title

    # If you want to use a specific title regardless of filename:
    # video_title_for_agent = "My Awesome Kids Video Test"
    video_title_for_agent = "Camp Camp: New Episodes March 1st!" # Using extracted filename as title for demo

    async def run_agent():
        summary = await video_safety_agent(local_video_path, video_title_for_agent)
        print("\n" + "="*50)
        print("Final Video Safety Report for Parents:")
        print("="*50)
        print(summary)
        print("="*50)

    # Run the asynchronous function
    asyncio.run(run_agent())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 2.3 MB/s eta 0:00:00


NameError: name 'YOUR_YOUTUBE_API_KEY' is not defined